In [ ]:
import os
import logging
import json
import datetime
import pickle
import telebot
from telebot import types
import pandas as pd
import schedule
import threading
import time
from datetime import datetime as dt
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
TELEGRAM_BOT_TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Initialize Telegram bot
bot = telebot.TeleBot(TELEGRAM_BOT_TOKEN)

# Scheduler Section
users_data = [
    {"chat_id": 850149704, "dz_bool": 0, "send_time": "07.03.2025, 18:45", "name": "Александр", "message_sent": False},
    {"chat_id": 426803001, "dz_bool": 1, "send_time": "07.03.2025, 18:45", "name": "Влад", "message_sent": False},
    {"chat_id": 1078826317, "dz_bool": 0, "send_time": "07.03.2025, 18:45", "name": "Руслана", "message_sent": False}
]
users_data_lock = threading.Lock()

def send_message_to_user(user):
    try:
        chat_id = user["chat_id"]
        name = user["name"]
        dz_bool = user["dz_bool"]
        
        if dz_bool == 1:
            message_text = f"Поздравляю, {name}, ты вошёл в 30% выполневших домашнее задание студентов нашего курса!"
        else:
            message_text = f"{name}, не забудьте отправить домашнее задание по теме: 'Я чемпион, я победитель, я TRUMP'."
        
        bot.send_message(chat_id, message_text)
        logger.info(f"Sent message to {chat_id} ({name})")
        user["message_sent"] = True
    except Exception as e:
        logger.error(f"Failed to send message to {chat_id}: {str(e)}")

def check_and_send_messages():
    current_time = dt.now().strftime("%d.%m.%Y, %H:%M")
    logger.info(f"Checking scheduled messages at {current_time}")
    
    with users_data_lock:
        for user in users_data:
            if user["send_time"] == current_time and not user["message_sent"]:
                send_message_to_user(user)

def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(30)  # Check every 30 seconds

schedule.every(45).seconds.do(check_and_send_messages)

# Q&A Bot Section
passwords = pd.read_excel('Adjusted_Generated_Passwords_ConsBeh.xlsx', dtype='str')
passwords.fillna('', inplace=True)

# Database setup
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, openai_api_base='https://api.proxyapi.ru/openai/v1')
chroma_db = Chroma(persist_directory='DataBase', embedding_function=embedding_model)
documents = [Document(page_content=d) for d in chroma_db.get()['documents']]
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 10

# Session management
session_store = {}
if os.listdir('UserStates'):
    files = [os.path.join('UserStates', f) for f in os.listdir('UserStates') if os.path.isfile(os.path.join('UserStates', f))]
    latest_file = max(files, key=os.path.getctime)
    with open(latest_file, 'rb') as f:
        session_store = pickle.load(f)

def get_session_history(session_id: str, k=3) -> ChatMessageHistory:
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    if len(session_store[session_id].messages) > k*2:
        del session_store[session_id].messages[:2]
    return session_store[session_id]

# Handlers
@bot.message_handler(commands=['start'])
def start(message):
    try:
        bot.send_message(
            message.chat.id,
            "Привет! Я - ИИ-эксперт МШУ 'Сколково' по поведению потребителей! Я могу помочь Вам разобраться с материалами курса. Задавайте вопросы!"
            "Оставьте обратную связь: https://forms.yandex.ru/u/6729d6a4eb614697263f2079/"
        )
        user_id = str(message.chat.id)
        if user_id not in passwords['USER_ID'].values:
            bot.send_message(message.chat.id, "Для авторизации отправьте пароль")
    except Exception as e:
        logger.error(f"Start command error: {str(e)}")

@bot.message_handler(commands=['clear_history'])
def clear_history(message):
    try:
        session_store[message.chat.id] = ChatMessageHistory()
        bot.send_message(message.chat.id, "Память очищена!")
    except Exception as e:
        logger.error(f"Clear history error: {str(e)}")

def handle_verification(message):
    user_id = str(message.chat.id)
    response = message.text
    try:
        if (response in passwords['Password'].values) and (passwords[passwords['Password'] == response]['USER_ID'].values[0] == ''):
            ind = passwords[passwords['Password'] == response].index[0]
            passwords.at[ind, 'USER_ID'] = user_id
            bot.send_message(message.chat.id, "Авторизация успешна! Задавайте вопросы.")
        else:
            bot.send_message(message.chat.id, "Неправильный пароль. Попробуйте снова.")
    except Exception as e:
        logger.error(f"Auth error for user {user_id}: {str(e)}")

@bot.message_handler(func=lambda m: True, content_types=['text'])
def query(message):
    try:
        # user_id = str(message.chat.id)
        # if user_id not in passwords['USER_ID'].values:
        #     handle_verification(message)
        #     return
        
        thinking_msg = bot.reply_to(message, "⏳ Обрабатываю запрос...")
        result = request_processing(message)
        
        bot.delete_message(message.chat.id, thinking_msg.message_id)
        bot.send_message(message.chat.id, result['answer'])
        
        # Save history
        with open('ChatHistory/{}_history.json'.format(dt.now().strftime("%Y%m%d%H%M%S")), 'w') as f:
            json.dump(result, f)
        
        with open('UserStates/{}_session.pkl'.format(dt.now().strftime("%Y%m%d%H%M%S")), 'wb') as f:
            pickle.dump(session_store, f)
        
        passwords.to_excel('Adjusted_Generated_Passwords.xlsx', index=False)
    except Exception as e:
        logger.error(f"Query processing error: {str(e)}")
        bot.reply_to(message, "Произошла ошибка. Повторите попытку позже.")

def request_processing(message):
    try:
        global DATABASE
    
        chatbot_purpose = f"# Роль\nВы являетесь доктором наук в области когнитивной и поведенческой психологии с обширным опытом в исследованиях, ориентированных на бизнес. Вы пишете энциклопедические статьи по когнитивной, поведенческой и нейропсихологии, маркетингу, исследованию пользователей и потребителей, наддированию (nudging) и геймификации по запросу пользователя."
        skills = """## Навыки
        ### Навык 1: Написание статей
        - Пользователь делает запрос на русском или английском языке, а вы пишете длинную, подробную, хорошо структурированную статью на русском языке, опираясь исключительно на свои фрагменты памяти и предоставленный вам контекст из базы знаний.
        - Сохраняйте текст фрагментов памяти и контекст из базы знаний как можно точнее, интегрируя фрагменты друг с другом и объединяя их для поддержания логического повествования и сохранения максимального количества оригинального текста.
        - Не выдумывайте информацию; интегрируйте только то, что доступно в вашем знании.
        - Дублируйте научные и бизнес-термины (отдельные слова или устойчивые фразы) на английском языке в скобках.
    
        ### Навык 2: Структурирование информации
        - Сначала суммируйте все фрагменты памяти в виде структурированного плана (аналогично плану википедийной статьи), обязательно включив разделы "Исследовательские направления" и "Ключевые слова".
        - Затем напишите статью, объединяя фрагменты памяти так, чтобы ни один фрагмент не был утерян.
        - При необходимости добавьте ссылки в стиле Чикаго.
    
        ### Навык 3: Исследовательские перспективы
        - В предпоследнем абзаце статьи приведите список потенциальных исследовательских направлений, вопросов и тем для будущих исследований, вытекающих из содержания статьи.
        - Игнорируйте этические аспекты, сосредоточившись на экспериментальных (лабораторных и полевых) исследованиях.
    
        ### Навык 4: Ключевые слова
        - В конце статьи укажите список ключевых слов и фраз, разделенных запятыми (не в виде списка).
    """
    
        constraints1 = '''## Ограничения:
    - Используйте только информацию, доступную в вашем контексте; не добавляйте вымышленную информацию.
    - В конце процесса всегда убедитесь, что вы написали полную статью, а не только ее план. Также проверьте, что статья написана на русском языке. Убедитесь, что статья начинается с плана, а основной текст следует за ним.
    - Все статьи всегда должны быть написаны на русском языке, независимо от языка запроса.'''
    
        system_prompt = chatbot_purpose + skills +constraints1
        ensemble_retriever = DATABASE.as_retriever(search_kwargs={'k': 15})
    
        model = ChatOpenAI(model='gpt-4o-mini', max_tokens=1024, api_key=openai_api_key,
                           base_url='https://api.proxyapi.ru/openai/v1')
        contextualize_q_system_prompt = (
            "Для ответа на вопросы пользователя учитывай историю чата и последний вопрос пользователя"
        )
        contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt + contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
    
        history_aware_retriever = create_history_aware_retriever(
            model, ensemble_retriever, contextualize_q_prompt
        )
    
        system_prompt_new = (
            "Ты персона, которая описывается ниже:\n" + system_prompt + "Пользователь хочет знать:\n{input}\n.Используй следующие фрагменты извлеченного контекста из твоей базы знаний, чтобы максимально точно ответить на вопрос \n{context}\n"
        )
    
        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt_new),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
    
        question_answer_chain = create_stuff_documents_chain(model, qa_prompt)
        rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
        conversational_rag_chain = RunnableWithMessageHistory(
            rag_chain,
            get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer",
        )
    
        inv = conversational_rag_chain.invoke(
            {"input": message.text},
            config={
                "configurable": {"session_id": message.chat.id}
            },
        )
        return inv
    except Exception as e:
        logger.error("Request processing failed: {}".format(str(e)))
        return {"answer": "Ошибка обработки запроса"}

def main():
    logger.info("Запуск глобального бота...")
    
    # Инициализация планировщика
    scheduler_thread = threading.Thread(target=run_scheduler)
    scheduler_thread.daemon = True
    scheduler_thread.start()

    while True:
        try:
            logger.info("Начало поллинга бота...")
            bot.polling(none_stop=True, interval=3, timeout=60)
        except Exception as e:
            logger.error(f"Ошибка поллинга бота: {str(e)}")
            time.sleep(10)
        else:
            logger.info("Поллинг бота завершен штатно")

if __name__ == "__main__":
    main()